In [69]:
def run_completion(chat_method, user_query):
    print(chat_method)
    system_prompt = "You are a helpful assistant."
    functions = [
      {
          "type": "function",
        "function": {
          "name": "getCurrentWeather",
          "description": "Get the weather in location",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
              "unit": {"type": "string", "enum": ["c", "f"]}
            },
            "required": ["location"]
          }
        }
      },
      {    "type": "function",
        "function":
        {
            "name": "orderUmbrella",
            "description": "Do this to help user to order an umbrella online", 
            "parameters": {
                "type": "object",
                "properties": {
                    "number_to_buy": {
                        "type": "integer",
                        "description": "the amount of umbrellas to buy"
                    }
                },
                "required": [
                    "number_to_buy"
                ]
            }
        }},
    ]
    # functions = [{"type": "function","function":{"name":"calculate_distance","description":"Calculate the distance between two locations","parameters":{"type":"object","properties":{"origin":{"type":"string","description":"The starting location"},"destination":{"type":"string","description":"The destination location"},"mode":{"type":"string","description":"The mode of transportation"}},"required":["origin","destination","mode"]}}},{"type": "function","function":{"name":"generate_password","description":"Generate a random password","parameters":{"type":"object","properties":{"length":{"type":"integer","description":"The length of the password"}},"required":["length"]}}}]

    msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}]

    res = chat_method(user_query, "gpt-4-0125-preview", functions=functions, msgs=msgs)
    print(f"First Response:")
    for tool_call in res.message.tool_calls:
        print(f"Tool Call: {tool_call.id}, {tool_call.function}")
    assistant_message = res.message
    tool_calls = []
    for tool_call in assistant_message.tool_calls:
        tool_calls.append( {
                            "id": tool_call.id,
                            "function": {"name": tool_call.function.name,
                                        "arguments": tool_call.function.arguments},
                            "type": "function",
                        })
    msgs.append({"role": "assistant",  "tool_calls": tool_calls})
    
    for i, tool_call in enumerate(assistant_message.tool_calls):
        if tool_call.function.name == "getCurrentWeather":
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"temprature is {i * 50} degree"})
        else:
            msgs.append({"role": "tool", "tool_call_id": str(assistant_message.tool_calls[i].id), "name": assistant_message.tool_calls[i].function.name, "content": f"Order placed."})
        

    print("Print before second response...")
    res_next = chat_method(user_query, "gpt-4-0125-preview", functions=functions, msgs=msgs)
    for m in msgs:
        print(m)
    print(f"Second Response: {res_next.message}")

## OpenAI

In [70]:
import openai


def get_oai_response(prompt, model, functions, msgs):
  openai.api_key = "sk-" ## Add your API key here
  openai.base_url = "https://api.openai.com/v1/"
  
  try:
    completion = openai.chat.completions.create(
      model=model,
      temperature=0.1,
      messages=msgs,
      tools=functions,
      tool_choice="auto",
      # functions=functions,
      # function_call="auto",
      stream=False,
    )
    return completion.choices[0]
  except Exception as e:
    print(e, model, prompt)


In [71]:
# user_query = "What is the distance between San Francisco and Cupertino by car and by air"
# user_query = "weather in boston as well as cupertino?"
user_query = "order 2 umbrellas"
run_completion(get_oai_response, user_query)

<function get_oai_response at 0x10b9c93f0>
First Response:
Tool Call: call_FYLEpX5CVo2dqSyNupcgtFak, Function(arguments='{"number_to_buy":2}', name='orderUmbrella')
Print before second response...
{'role': 'system', 'content': 'You are a helpful assistant.'}
{'role': 'user', 'content': 'order 2 umbrellas'}
{'role': 'assistant', 'tool_calls': [{'id': 'call_FYLEpX5CVo2dqSyNupcgtFak', 'function': {'name': 'orderUmbrella', 'arguments': '{"number_to_buy":2}'}, 'type': 'function'}]}
{'role': 'tool', 'tool_call_id': 'call_FYLEpX5CVo2dqSyNupcgtFak', 'name': 'orderUmbrella', 'content': 'Order placed.'}
Second Response: ChatCompletionMessage(content="I've placed the order for 2 umbrellas for you. Is there anything else I can help with?", role='assistant', function_call=None, tool_calls=None)


## Function.cpp

In [72]:
import openai


def get_mistral_rubra_response(prompt, model, functions, msgs):
  openai.api_key = "sk-"
  openai.base_url = "http://localhost:8019/v1/"
  
  try:
    completion = openai.chat.completions.create(
      model=model,
      temperature=0.1,
      messages=msgs,
      tools=functions,
      tool_choice="auto",
      stream=False,
    )
    return completion.choices[0]
  except Exception as e:
    print(e, model, prompt)



In [73]:
# user_query = "generate a password of length 10 and another of length 20" 
# user_query = "what's the weather in Boston and Cupertino?"
user_query = "order 2 umbrellas"
run_completion(get_mistral_rubra_response, user_query)

<function get_mistral_rubra_response at 0x10b9cac20>
First Response:
Tool Call: 0, Function(arguments='{"number_to_buy":"2"}', name='orderUmbrella')
Print before second response...
{'role': 'system', 'content': 'You are a helpful assistant.'}
{'role': 'user', 'content': 'order 2 umbrellas'}
{'role': 'assistant', 'tool_calls': [{'id': '0', 'function': {'name': 'orderUmbrella', 'arguments': '{"number_to_buy":"2"}'}, 'type': 'function'}]}
{'role': 'tool', 'tool_call_id': '0', 'name': 'orderUmbrella', 'content': 'Order placed.'}
Second Response: ChatCompletionMessage(content=' Your order for 2 umbrellas has been placed.', role='assistant', function_call=None, tool_calls=None)


#### IGNORE the following for now.

In [46]:
system_prompt = "You are a helpful assistant."
functions = [
    {"function":
    {
        "name": "get_stock_fundermentals",
        "description": "Get the stock fundermentals data",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol, e.g. AAPL, GOOG"
                }
            },
            "required": [
                "symbol"
            ]
        }
    }},
    {"function":{
        "name": "check_word_anagram",
        "description": "Check if two words are anagrams of each other",
        "parameters": {
            "type": "object",
            "properties": {
                "word1": {
                    "type": "string",
                    "description": "The first word"
                },
                "word2": {
                    "type": "string",
                    "description": "The second word"
                }
            },
            "required": [
                "word1",
                "word2"
            ]
        }
    }}
]

user_query = "What's the stock fundementals of Tesla and google"

# 
# msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}, {"role": "function", "content": '<<functions>>[get_stock_price(symbol="TSLA")], <<functions>>[get_stock_price(symbol="GOOG")]'}, {"role": "observation", "content": "{'symbol': 'TSLA', 'company_name': 'Tesla, Inc.', 'sector': 'Consumer Cyclical', 'industry': 'Auto Manufacturers', 'market_cap': 611384164352, 'pe_ratio': 49.604652, 'pb_ratio': 9.762013, 'dividend_yield': None, 'eps': 4.3, 'beta': 2.427, '52_week_high': 299.29, '52_week_low': 152.37}}"}]
msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query},]
res = get_mistral_rubra_response(user_query, "mistral_rubra", functions=functions, msgs=msgs)
print(res.message.content)

None


In [47]:
import json
import re
import ast

content = "<<functions>>[get_current_weather(location='Boston, MA', api_key=123456789, unit='fahrenheit'), func(x= 1, b='2', c=123)]"
regex = re.compile(r"<<functions>>\[(.*?)\]", re.DOTALL)
matches = re.findall(regex, content)

print("content:", content)

def parse_function_call(call):
    func_name, args_str = call.split('(', 1)
    args_str = args_str.rstrip(')')
    args_list = args_str.split(',')
    args_dict = {}
    for arg in args_list:
        key, value = arg.split('=')
        key = key.strip()
        value = value.strip()
        try:
            # Use ast.literal_eval to safely parse the string to its Python type
            parsed_value = ast.literal_eval(value)
        except ValueError as e:
            # If parsing fails, keep the original string. 
            # This might happen if the value is a string that's not quoted as a Python literal.
            print(f"Error parsing value {value}: {e}")
            parsed_value = value
        args_dict[key] = parsed_value
    return {"name": func_name.strip(), "arguments": args_dict}

result_dicts = []
for match in matches:
    # Splitting each function call from the match. We add ')' back because it was used as a delimiter
    function_calls = [f"{func})" for func in match.split('),') if func]
    print(function_calls)
    for function_call in function_calls:
        # Removing the trailing ')' that was added for the last function call
        if function_call.endswith(')'):
            function_call = function_call[:-1]
        result_dict = parse_function_call(function_call.strip())
        result_dicts.append(result_dict)
    print(result_dicts)

res = json.dumps(result_dicts, ensure_ascii=False)
res

content: <<functions>>[get_current_weather(location='Boston, MA', api_key=123456789, unit='fahrenheit'), func(x= 1, b='2', c=123)]
["get_current_weather(location='Boston, MA', api_key=123456789, unit='fahrenheit')", " func(x= 1, b='2', c=123))"]


SyntaxError: unterminated string literal (detected at line 1) (<unknown>, line 1)

In [ ]:
import ast

raw_input_str = "<<functions>>[get_current_weather(location='Boston, MA', api_key=123456789, unit='fahrenheit'), func('cde', x= 1, b='2', c=[1, 2, {'a': 1, 'b': 2}])]"
# raw_input_str = "<<functions>>[get_current_weather(location='Boston, MA', api_key=123456789, unit='fahrenheit'), func( x=1, b='2', c=123)]"
input_str = raw_input_str.split('<<functions>>')[1]
# Parse the string into an AST
parsed_ast = ast.parse(input_str, mode='eval')

def find_calls(node):
    calls = []
    if isinstance(node, ast.Call):  # If it's a function call
        calls.append(node)
        return calls
    for child in ast.iter_child_nodes(node):
        calls.extend(find_calls(child))
    return calls

# Extract all function call nodes
calls = find_calls(parsed_ast.body)

functions = []
for call in calls:
    if isinstance(call.func, ast.Name):  # Ensure it's a named function
        function_name = call.func.id
        args = [ast.literal_eval(arg) for arg in call.args]  # Convert all positional arguments
        kwargs = {kw.arg: ast.literal_eval(kw.value) for kw in call.keywords}  # Convert all keyword arguments
        functions.append({"name": function_name, "args": args, "kwargs":kwargs})

print(functions)


[{'name': 'get_current_weather', 'args': [], 'kwargs': {'location': 'Boston, MA', 'api_key': 123456789, 'unit': 'fahrenheit'}}, {'name': 'func', 'args': ['cde'], 'kwargs': {'x': 1, 'b': '2', 'c': [1, 2, {'a': 1, 'b': 2}]}}]


In [ ]:
import ast

input_str = "[get_current_weather(location='Boston, MA', api_key=123456789, unit='fahrenheit'), func('cde', x= 1, b='2', c=[func_nested(1, 2), {'a': func_deep('value')}])]"

def ast_node_to_object(node):
    if isinstance(node, ast.Constant):
        return node.value
    elif isinstance(node, ast.List):
        return [ast_node_to_object(n) for n in node.elts]
    elif isinstance(node, ast.Dict):
        return {ast_node_to_object(key): ast_node_to_object(value) for key, value in zip(node.keys, node.values)}
    elif isinstance(node, ast.Tuple):
        return tuple(ast_node_to_object(n) for n in node.elts)
    elif isinstance(node, ast.Call):
        return ast.unparse(node)
        # Handle function calls: convert to a representation with the function name and arguments
        # func_name = ast_node_to_object(node.func)  # Get the function name
        # args = [ast_node_to_object(arg) for arg in node.args]  # Convert all positional arguments
        # kwargs = {kw.arg: ast_node_to_object(kw.value) for kw in node.keywords}  # Convert all keyword arguments
        # return {"function": func_name, "args": args, "kwargs": kwargs}
    elif isinstance(node, ast.Name):
        return node.id  # Return the identifier name
    # Add more cases here as needed
    return None

# Parse the string into an AST
parsed_ast = ast.parse(input_str, mode='eval')

# Function to find only the top-level Call nodes
def find_top_level_calls(node):
    calls = []
    if isinstance(node, ast.Call):  # If it's a function call
        calls.append(node)
        # Do not descend into child nodes to ensure we're only capturing top-level calls
        return calls
    for child in ast.iter_child_nodes(node):
        # Recursively find calls without going into nested calls
        calls.extend(find_top_level_calls(child))
    return calls

# Extract all top-level function call nodes
top_level_calls = find_top_level_calls(parsed_ast.body)

# Process each call node to get the details you want
functions = []
for call in top_level_calls:
    if isinstance(call.func, ast.Name):  # Ensure it's a named function
        function_name = call.func.id
        args = [ast_node_to_object(arg) for arg in call.args]  # Convert all positional arguments
        kwargs = {kw.arg: ast_node_to_object(kw.value) for kw in call.keywords}  # Convert all keyword arguments
        functions.append((function_name, args, kwargs))

print(functions)


[('get_current_weather', [], {'location': 'Boston, MA', 'api_key': 123456789, 'unit': 'fahrenheit'}), ('func', ['cde'], {'x': 1, 'b': '2', 'c': ['func_nested(1, 2)', {'a': "func_deep('value')"}]})]
